In [6]:
import os
import numpy as np
import pandas as pd

from joblib import load
from tqdm.notebook import tqdm
from torch.utils import data

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from IPython.display import clear_output

import torch
from torch.utils import data

from src.Sparse_vector.sparse_vector import SparseVector
from src.data_preparation import get_train_test_dataset
from src.train_test import set_random_seed, train


In [7]:
from src.interpretation import cnn_interpretation_pipeline

In [3]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f"z_dna/hg38_dna/") if f"{chrom}_" in i])
    return "".join([load(f"z_dna/hg38_dna/{file}") for file in files])


chroms = [f"chr{i}" for i in list(range(1, 23)) + ["X", "Y", "M"]]
all_features = [
    i[:-4] for i in os.listdir("z_dna/hg38_features/sparse/") if i.endswith(".pkl")
]
groups = ["DNase-seq", "Histone", "RNA polymerase", "TFs and others"]
feature_names = [i for i in all_features]


In [4]:
%%time
DNA = {chrom: chrom_reader(chrom) for chrom in tqdm(chroms)}

ZDNA_data = load("Quad/g4.pkl")

DNA_features = {
    feature: load(f"z_dna/hg38_features/sparse/{feature}.pkl")
    for feature in tqdm(feature_names)
}


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1946 [00:00<?, ?it/s]

CPU times: user 2min 24s, sys: 15.6 s, total: 2min 39s
Wall time: 4min 27s


In [5]:
np.random.seed(10)

width = 100

train_dataset, test_dataset = get_train_test_dataset(width, chroms, feature_names, DNA, DNA_features, ZDNA_data)

100%|██████████| 165/165 [00:00<00:00, 38638.83it/s]
/opt/software/python/envs/google_colab_gpu_2024/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [6]:
params = {"batch_size": 1, "num_workers": 5, "shuffle": True, "pin_memory": True}

loader_train = data.DataLoader(train_dataset, **params)
loader_test = data.DataLoader(test_dataset, **params)


In [ ]:
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score
from IPython.display import clear_output


class ImageZ(nn.Module):
    def __init__(self, width, features_count):
        super().__init__()
        self.width = width
        self.features_count = features_count

        self.seq = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(2, 4),
            nn.Conv2d(4, 8, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(4, 8),
            nn.Conv2d(8, 16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(8, 16),
            nn.Conv2d(16, 32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(16, 32),
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(16, 64),
            nn.Conv2d(64, 128, kernel_size=(5, 5), padding=2),
            nn.ReLU(),
            nn.GroupNorm(32, 128),
            nn.Conv2d(128, 64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(32, 64),
            nn.Conv2d(64, 32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(16, 32),
            nn.Conv2d(32, 16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(8, 16),
            nn.Conv2d(16, 8, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(4, 8),
            nn.Conv2d(8, 4, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(4, 4),
            nn.Conv2d(4, 1, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.GroupNorm(1, 1),
            nn.AlphaDropout(p=0.2),
            nn.Linear(features_count + 4, 500),
            nn.AlphaDropout(p=0.2),
            nn.SELU(),
            nn.Linear(500, 2),
        )

    def forward(self, x):
        batch = x.shape[0]
        x = x.reshape(batch, 1, self.width, self.features_count + 4)
        x = self.seq(x)
        x = torch.squeeze(x)
        x = F.log_softmax(x, dim=-1)
        return x


In [23]:
import gc

model = ImageZ(width, len(feature_names))
model = nn.DataParallel(model)
model.load_state_dict(torch.load("quad_model_0.692.pt", weights_only=True))
model = model.to("cuda")
model.eval()

gc.collect()

torch.cuda.empty_cache()


In [ ]:
from torch.utils.data import DataLoader, Subset

subset_size = 10000
indices = list(range(subset_size))

subset = Subset(test_dataset, indices)
params = {"batch_size": 1, "num_workers": 5, "shuffle": True, "pin_memory": True}

loader_test_subset = data.DataLoader(subset, **params)


In [29]:
mean_IG = cnn_interpretation_pipeline(
    model,
    loader_test_subset,
    loader_train,
    width,
    "interpretation_files/mean_IntegratedGradients_quad",
    "IntegratedGradients",
    need_return=1,
)


In [30]:
mean_DL = cnn_interpretation_pipeline(
    model,
    loader_test_subset,
    loader_train,
    width,
    "interpretation_files/mean_DeepLift_quad",
    "DeepLift",
    need_return=1,
)


  0%|          | 0/500 [00:00<?, ?it/s]

/home/vvmalkov/.local/lib/python3.10/site-packages/captum/attr/_core/deep_lift.py:294: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  gradient_mask = apply_gradient_requirements(inputs_tuple)
/home/vvmalkov/.local/lib/python3.10/site-packages/captum/log/dummy_log.py:39: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  return func(*args, **kwargs)


Averaged tensor shape: torch.Size([1950])
Averaged tensor: tensor([-2.6325e-03,  1.3969e-02,  1.2204e-02,  ...,  4.2321e-07,
        -1.4544e-07, -1.7494e-06], dtype=torch.float64)
Interpretation result is an averaged tensor. It is saved as:
interpretation_files/mean_DeepLift_quad.pt


In [31]:
mean_IXG = cnn_interpretation_pipeline(
    model,
    loader_test_subset,
    loader_train,
    width,
    "interpretation_files/mean_InputXGradient_quad",
    "InputXGradient",
    need_return=1,
)


  0%|          | 0/10000 [00:00<?, ?it/s]

/home/vvmalkov/.local/lib/python3.10/site-packages/captum/attr/_core/input_x_gradient.py:118: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  gradient_mask = apply_gradient_requirements(inputs_tuple)


Averaged tensor shape: torch.Size([1950])
Averaged tensor: tensor([ 6.2885e-02,  7.4319e-01,  6.0160e-01,  ..., -1.0578e-04,
         1.1955e-04, -3.9027e-04], dtype=torch.float64)
Interpretation result is an averaged tensor. It is saved as:
interpretation_files/mean_InputXGradient_quad.pt


In [32]:
mean_GB = cnn_interpretation_pipeline(
    model,
    loader_test_subset,
    loader_train,
    width,
    "interpretation_files/mean_GuidedBackpropagation_quad",
    "GuidedBackpropagation",
    need_return=1,
)


  0%|          | 0/10000 [00:00<?, ?it/s]

/home/vvmalkov/.local/lib/python3.10/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:63: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  gradient_mask = apply_gradient_requirements(inputs_tuple)
/home/vvmalkov/.local/lib/python3.10/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:66: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


Averaged tensor shape: torch.Size([1950])
Averaged tensor: tensor([0.0013, 0.0518, 0.1676,  ..., 0.0014, 0.0007, 0.0004],
       dtype=torch.float64)
Interpretation result is an averaged tensor. It is saved as:
interpretation_files/mean_GuidedBackpropagation_quad.pt


In [33]:
mean_GS = cnn_interpretation_pipeline(
    model,
    loader_test_subset,
    loader_train,
    width,
    "interpretation_files/mean_GradientShap_quad",
    "GradientShap",
    need_return=1,
)


  0%|          | 0/500 [00:00<?, ?it/s]

Averaged tensor shape: torch.Size([1950])
Averaged tensor: tensor([-3.1518e-02,  1.4470e-01,  2.3712e-01,  ...,  2.6487e-06,
         3.1148e-06, -2.3233e-05], dtype=torch.float64)
Interpretation result is an averaged tensor. It is saved as:
interpretation_files/mean_GradientShap_quad.pt


In [34]:
# in our features data first 4 indices correspond to ACTG
mean_IG = mean_IG[4:]
mean_DL = mean_DL[4:]
mean_GS = mean_GS[4:]
mean_GB = mean_GB[4:]
mean_IXG = mean_IXG[4:]

In [35]:
features_weights = {
    "mean_IG": mean_IG,
    "mean_DL": mean_DL,
    "mean_GS": mean_GS,
    "mean_GB": mean_GB,
    "mean_IXG": mean_IXG,
}
features_weights = pd.DataFrame(features_weights)


In [36]:
features_weights.head()

,mean_IG,mean_DL,mean_GS,mean_GB,mean_IXG
0,-0.029835,8.217483e-06,-0.000157,0.046790,-1.184282e-02
1,-0.000036,8.201226e-07,0.000088,0.028352,-3.702821e-07
2,-0.028408,-7.423853e-05,0.006172,0.055155,-3.017598e-02
3,0.001172,-9.900524e-08,-0.000015,0.004410,6.141821e-04
4,-0.000030,4.661138e-07,-0.000075,0.006531,9.155139e-06


In [39]:
features = [i[:-4] for i in os.listdir('z_dna/hg38_features/sparse/') if i.endswith('.pkl')]
feature_names = [i for i in features]

features_weights["feature_names"] = feature_names

In [41]:
metric_cols = ['mean_IG', 'mean_DL', 'mean_GS', 'mean_GB', 'mean_IXG']


ranks = features_weights[metric_cols].rank(ascending=False, method='average')
features_weights['borda_rank'] = ranks.sum(axis=1)

In [42]:
features_weights.to_csv("interpretation_files/features_importance_G4.csv")